# Install Dependencies

In [1]:
!pip -q install datasets
!pip -q install unidecode
!pip -q install pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget -qq https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_examples.parquet
# !wget https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_products.parquet

In [2]:
import datasets
from unidecode import unidecode
import csv
import datetime
import pandas as pd

datasets.logging.set_verbosity_error()

# Simple POC

In [ ]:
dataset = datasets.load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    f"raw_meta_CDs_and_Vinyl",
    split='full',
    streaming=True,
    trust_remote_code=True,
)

In [ ]:
a = dataset.filter(lambda x: x['title'] not in (None, "", "None") and len(x['title']) <= 100)
a = a.take(10)

In [ ]:
for idx, i in enumerate(a):
    print(len(i['title']), '\t', i['title'])

20 	 Release Some Tension
9 	 Rio Angie
12 	 Lost in Love
17 	 Somewhere in Time
15 	 Kimmon Waldruff
17 	 Made in Argentina
24 	 Heavy Hearts, Hard Fists
42 	 Johnny Hodges With Lawrence Welk Orchestra
12 	 Song Painter
32 	 Froberger: Works For Harpsichord


In [ ]:
import random

postalcodes = [r for r in csv.reader(open('postcodes.csv'))]
random_postalcode = random.choice(postalcodes)
LATITUDE = random_postalcode[-2]
LONGITUDE = random_postalcode[-1]
POSTAL_CODE = random_postalcode[0]

print(random_postalcode, float(LATITUDE), float(LONGITUDE), POSTAL_CODE)

['V8R3V1', 'VICTORIA', 'BC', '8', '48.439395', '-123.343512'] 48.439395 -123.343512 V8R3V1


# Write `query.csv`

In [ ]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet('./shopping_queries_dataset_examples.parquet')

# Display the first few rows
print(df.head())

   example_id           query  query_id  product_id product_locale esci_label  \
0           0   revent 80 cfm         0  B000MOO21W             us          I   
1           1   revent 80 cfm         0  B07X3Y6B1V             us          E   
2           2   revent 80 cfm         0  B07WDM7MQQ             us          E   
3           3   revent 80 cfm         0  B07RH6Z8KW             us          E   
4           4   revent 80 cfm         0  B07QJ7WYFQ             us          E   

   small_version  large_version  split  
0              0              1  train  
1              0              1  train  
2              0              1  train  
3              0              1  train  
4              0              1  train  


In [ ]:
print(df.columns)

Index(['example_id', 'query', 'query_id', 'product_id', 'product_locale',
       'esci_label', 'small_version', 'large_version', 'split'],
      dtype='object')


In [ ]:
locale_us = df.loc[df['product_locale'] == 'us']
unique_query = locale_us['query'].unique()

FIELDS = ['query']

csv_file = open('./query.csv', 'w+')
writer = csv.DictWriter(csv_file, fieldnames=FIELDS)
writer.writeheader()

for q in unique_query:
    writer.writerows([{'query': f'{unidecode(q)}'}])

csv_file.close()

# Write `listings.csv`

In [4]:
# from random import randrange
import random

# Number of records per category.
NUM_RECORDS = 15_000

# Constants
FIELDS = ['seller_id', 'title', 'price', 'latitude', 'longitude', 'postal_code', 'status', 'listed_at', 'last_updated_at', 'category']
STATUS = 'AVAILABLE'
TIMESTAMP=datetime.datetime(2024, 7, 20)

postalcodes = [r for r in csv.reader(open('postcodes.csv'))]

# from https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw/meta_categories
categories = [
    'Amazon_Fashion',
    'Appliances',
    'Arts_Crafts_and_Sewing',
    'Automotive',
    'CDs_and_Vinyl',
    'Cell_Phones_and_Accessories',
    'Clothing_Shoes_and_Jewelry',
    'Electronics',
    'Gift_Cards',
    'Handmade_Products',
    'Health_and_Household',
    'Home_and_Kitchen',
    'Industrial_and_Scientific',
    'Musical_Instruments',
    'Office_Products',
    'Patio_Lawn_and_Garden',
    'Sports_and_Outdoors',
    'Tools_and_Home_Improvement',
    'Toys_and_Games',
    'Video_Games'
]

# Create csv file - this will overwrite existing files.
csv_file = open('./listings.csv', 'w+')

writer = csv.DictWriter(csv_file, fieldnames=FIELDS)
writer.writeheader()

for category in categories:

    # stream dataset
    dataset = datasets.load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        f"raw_meta_{category}",
        split="full",
        trust_remote_code=True,
        streaming=True
    )

    dataset = dataset.filter(lambda x: x['title'] not in (None, '', 'None') \
                             and len(unidecode(x['title'])) <= 100 \
                             and x['main_category'] not in (None, '', 'None'))
    dataset = dataset.take(NUM_RECORDS)

    print(f"Writing [{category}]...\n")
    for i, item in enumerate(dataset):

        random_postalcode = random.choice(postalcodes)
        LATITUDE = random_postalcode[-2]
        LONGITUDE = random_postalcode[-1]
        POSTAL_CODE = random_postalcode[0]

        try:
            item['price'] = float(item['price'])
            LATITUDE = float(LATITUDE)
            LONGITUDE = float(LONGITUDE)
        except:
            item['price'] = 499.03
            LATITUDE = 48.444051
            LONGITUDE = -123.338038
            POSTAL_CODE = 'V8T2C5'

        row = [{
                'seller_id': random.randrange(1, 4021),
                'title': unidecode(item['title']),
                'price': item['price'],
                'latitude': LATITUDE,
                'longitude': LONGITUDE,
                'postal_code': POSTAL_CODE,
                'status': STATUS,
                'listed_at': TIMESTAMP,
                'last_updated_at': TIMESTAMP,
                'category': item['main_category']
            }]

        writer.writerows(row)
csv_file.close()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Writing [Amazon_Fashion]...

Writing [Appliances]...

Writing [Arts_Crafts_and_Sewing]...

Writing [Automotive]...

Writing [CDs_and_Vinyl]...

Writing [Cell_Phones_and_Accessories]...

Writing [Clothing_Shoes_and_Jewelry]...

Writing [Electronics]...

Writing [Gift_Cards]...

Writing [Handmade_Products]...

Writing [Health_and_Household]...

Writing [Home_and_Kitchen]...

Writing [Industrial_and_Scientific]...

Writing [Musical_Instruments]...

Writing [Office_Products]...

Writing [Patio_Lawn_and_Garden]...

Writing [Sports_and_Outdoors]...

Writing [Tools_and_Home_Improvement]...

Writing [Toys_and_Games]...

Writing [Video_Games]...



# Download file

In [5]:
from google.colab import files

files.download('listings.csv')
# files.download('query.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>